In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [4]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '#####' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [5]:
annotation_types_points = {}

for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

annotation_types_points

{384: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 384,
  'name': 'L0',
  'node_count': 0,
  'product': 104,
  'sort_order': 0,
  'vector_type': 1},
 385: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 385,
  'name': 'L1',
  'node_count': 0,
  'product': 104,
  'sort_order': 1,
  'vector_type': 1},
 386: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 386,
  'name': 'L2',
  'node_count': 0,
  'product': 104,
  'sort_order': 2,
  'vector_type': 1},
 387: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,

In [6]:
annotations = []

for image_set_name in tqdm(["CCMCT_2.0HT", "CCMCT_Aperio", "CCMCT_Axio", "CCMCT_S210", 
                       "Cyto_2.0HT", "Cyto_Aperio", "Cyto_Axio", "Cyto_S210"]):
    
    scanner = image_set_name.split("_")[1]
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:

        image = images_api.retrieve_image(id=image_id)
        
        image_width, image_height = image.width, image.height

        for anno in annotations_api.list_annotations(image=image_id, pagination=False, # BoundBox ID
                                                     deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:

            image_type = "CCMCT" if "CCMCT" in image_set_name else "Cyto"
            
            if anno.annotation_type in annotation_types_points:
            
                anno.annotation_type = annotation_types_points[anno.annotation_type]
                annotations.append([scanner, image.id, image.name, image_type, image_width, image_height, 
                                    anno.id, anno.vector, anno.unique_identifier, anno.annotation_type.id, 
                                    anno.annotation_type.name])


annotations = pd.DataFrame(annotations, columns=["scanner", "image_id", "image_name", "image_type", "image_width", "image_height",
                                                 "id", "vector", "unique_identifier", "annotation_type", "type_name"])
annotations.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:18<00:00,  2.36s/it]


,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,type_name
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4


In [7]:
annotations.to_csv("GT.csv", index=False)

In [9]:
df = annotations[(annotations["image_type"] == "CCMCT") & 
                    (annotations["type_name"] == "L20")]

df

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,type_name
20,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755426,"{'x1': 128606, 'x2': 128660, 'y1': 16173, 'y2'...",1f8ca805-a23e-4eec-8aad-8bec244fc890,407,L20
45,2.0HT,10404,N2_CCMCT_518711B_1.ndpi,CCMCT,122880,96768,2755451,"{'x1': 100721, 'x2': 100775, 'y1': 14803, 'y2'...",efbb7806-adfc-4f92-be3a-208391499fc1,407,L20
69,2.0HT,10402,N2_CCMCT_183715A_1.ndpi,CCMCT,86016,75008,2755476,"{'x1': 71178, 'x2': 71232, 'y1': 10794, 'y2': ...",294c4a75-4ba2-444f-9792-8f4ba22549c5,407,L20
139,Aperio,10416,A_CCMCT_518711B_1.svs,CCMCT,107999,83794,2755301,"{'x1': 88792, 'x2': 88842, 'y1': 13945, 'y2': ...",efbb7806-adfc-4f92-be3a-208391499fc1,407,L20
164,Aperio,10415,A_CCMCT_380609B_1.svs,CCMCT,125999,89356,2755326,"{'x1': 112217, 'x2': 112267, 'y1': 12091, 'y2'...",1f8ca805-a23e-4eec-8aad-8bec244fc890,407,L20
188,Aperio,10401,A_CCMCT_183715A_1.svs,CCMCT,77399,66558,2755351,"{'x1': 63177, 'x2': 63227, 'y1': 9866, 'y2': 9...",294c4a75-4ba2-444f-9792-8f4ba22549c5,407,L20
282,Axio,10412,Z_CCMCT_518711B_1.tif,CCMCT,119809,93153,2755576,"{'x1': 98936, 'x2': 98992, 'y1': 13460, 'y2': ...",efbb7806-adfc-4f92-be3a-208391499fc1,407,L20
306,Axio,10409,Z_CCMCT_183715A_1.tif,CCMCT,85167,72527,2755601,"{'x1': 70670, 'x2': 70726, 'y1': 8618, 'y2': 8...",294c4a75-4ba2-444f-9792-8f4ba22549c5,407,L20
376,S210,10413,N1_CCMCT_518711B_1.ndpi,CCMCT,122880,98560,2755676,"{'x1': 102275, 'x2': 102331, 'y1': 17241, 'y2'...",efbb7806-adfc-4f92-be3a-208391499fc1,407,L20
401,S210,10411,N1_CCMCT_380609B_1.ndpi,CCMCT,153600,112640,2755701,"{'x1': 130955, 'x2': 131011, 'y1': 18371, 'y2'...",1f8ca805-a23e-4eec-8aad-8bec244fc890,407,L20


In [12]:
annotations_api.list_annotations(image=10410, pagination=False, deleted=False, fields="annotation_type,id,image,vector,unique_identifier,deleted").results

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': False,
  'description': None,
  'id': 2755406,
  'image': 10410,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '95183f6e-0a00-4de4-9c97-fedfec655a27',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': 16806},
  'verified_by_user': None},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': False,
  'description': None,
  'id': 2755407,
  'image': 10410,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'cc3def2b-5631-4619-ab08-51f2a69c667d',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': 35502},
  'verified_by_user': None},
 {'annotation_type': 386,
  'annotationversion_set': [],
  'blurred': 